In [1]:
#!/usr/bin/env python

import torch

from torchvision import datasets
import torchvision.transforms as transforms
import torch.nn as nn
# from torch import nn, optim
from torch.autograd import Variable
from   torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
from torchtext import data, datasets
from torchtext.data import Field, BucketIterator, TabularDataset
from pytorchtools import EarlyStopping
# from utils import Logger
from torchsummary import summary

import MMD as mmd
import phate
import umap
from sklearn.decomposition import PCA
import scprep
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn.metrics import r2_score
from numpy import linalg as LA
phate_op = phate.PHATE()

import os
import numpy as np
import pandas as pd
from random import sample 
import matplotlib.pyplot as plt
import seaborn
from itertools import islice
seaborn.set_context(context="talk")
import math, copy, time
import sys

# class DiscriminatorNet(nn.Module):
#     """
#     A three hidden-layer discriminative neural network
#     """
#     def __init__(self):
#         super(DiscriminatorNet, self).__init__()
#         n_features = 512
#         n_out = 1
        
#         self.hidden0 = nn.Sequential( 
#             nn.Linear(n_features, 100),
#             nn.LeakyReLU(0.2)
#         )
#         self.hidden1 = nn.Sequential(
#             nn.Linear(100, 100),
#             nn.LeakyReLU(0.2)
#         )
#         self.hidden2 = nn.Sequential(
#             nn.Linear(100, 100),
#             nn.LeakyReLU(0.2)
#         )
#         self.out = nn.Sequential(
#             torch.nn.Linear(100, n_out),
#             torch.nn.Sigmoid()
#         )

#     def forward(self, x):
#         x = self.hidden0(x)
#         x = self.hidden1(x)
#         x = self.hidden2(x)
#         x = self.out(x)
#         return x

# class GeneratorNet(torch.nn.Module):
#     """
#     A three hidden-layer generative neural network
#     """
#     def __init__(self):
#         super(GeneratorNet, self).__init__()
#         n_features = n_z
#         n_out = 784
        
#         self.hidden0 = nn.Sequential(
#             nn.Linear(n_features, 100),
#             nn.LeakyReLU(0.2)
#         )
#         self.hidden1 = nn.Sequential(            
#             nn.Linear(100, 100),
#             nn.LeakyReLU(0.2)
#         )
#         self.hidden2 = nn.Sequential(
#             nn.Linear(100, 100),
#             nn.LeakyReLU(0.2)
#         )
        
#         self.out = nn.Sequential(
#             nn.Linear(100, n_out),
#             nn.Tanh()
#         )

#     def forward(self, x):
#         x = self.hidden0(x)
#         x = self.hidden1(x)
#         x = self.hidden2(x)
#         x = self.out(x)
#         return x

def images_to_vectors(images):
    return images.view(images.size(0), 784)

def vectors_to_images(vectors):
    return vectors.view(vectors.size(0), 1, 28, 28)

def noise(size):
    '''
    Generates a 1-d vector of gaussian sampled random values
    '''
    n = Variable(torch.randn(size, n_z))
    return n

# def train_discriminator(optimizer, french_data, english_data):
#     # Reset gradients
#     optimizer.zero_grad()

#     # Train on french data
#     loss_discriminator = 0
#     for j in range(1, french_data.size(0)):
#         # print("Target:", end="\t")
#         # for k in range(1, english_data.size(1)): #Maximum sentence length in this batch
#         #     sym = TGT.vocab.itos[english_data[j,k]] #Translating indexes to words of the first english sentence in this batch.
#         #     if sym == "</s>": break
#         #     print("(j={},k={}) {}".format(j,k,sym))
#         # print()
#         # print("Source:", end="\t")
#         # for k in range(1, french_data.size(1)):
#         #     sym = SRC.vocab.itos[french_data[j,k]] #Translating indexes to words of the first english sentence in this batch.
#         #     if sym == "</s>": break
#         #     print("(j={},k={}) {}".format(j,k,sym))
#         # print()

#         french_data2 = french_data[j,french_data[j,:]!=1] #1=<blank>
#         english_data2 = english_data[j,english_data[j,:]>3] #1=<blank>, 2=<s>, 3=</s>
#         embed_src = model.src_embed[0].lut(french_data2.to(device)).cpu().detach()
#         embed_tgt = model.tgt_embed[0].lut(english_data2.to(device)).cpu().detach()

#         prediction_french = discriminator(embed_src)
#         prediction_english = discriminator(embed_tgt)
#         # print("prediction_french.size(): {}".format(prediction_french.size()))
#         # print("embed_src: {}".format(embed_src))
#         # print("embed_src.size(): {}".format(embed_src.size()))
#         # print("prediction_english.size(): {}".format(prediction_english.size()))
#         # print("embed_tgt: {}".format(embed_tgt))
#         # print("embed_tgt.size(): {}".format(embed_tgt.size()))
#         # print("french_data.squeeze().size(0)): {}".format(french_data.squeeze().size(0)))
        
#         # Calculate error and backpropagate
#         loss_discriminator += -(torch.mean(prediction_french) - torch.mean(prediction_english))**2
#     loss_discriminator = (loss_discriminator/french_data.size(0))*weight_loss_discriminator

#     loss_discriminator.backward()
#     optimizer.step()

#     return loss_discriminator, torch.mean(prediction_french), torch.mean(prediction_english)

def french_data_target(size):
    '''
    Tensor containing ones, with shape = size
    '''
    data = Variable(torch.ones(size, 1))
    # if torch.cuda.is_available(): return data.cuda()
    return data

def english_data_target(size):
    '''
    Tensor containing zeros, with shape = size
    '''
    data = Variable(torch.zeros(size, 1))
    # if torch.cuda.is_available(): return data.cuda()
    return data

def plot_embeddings(path_save, string_name, epoch):
    
    #check it folder exists
    directory = os.path.join(path_save,string_name)
    if not os.path.exists(directory):
        print("creating {}".format(directory))
        os.makedirs(directory)
    
    n_samples = 7000

    sent_tgt = TGT.vocab.itos
    sent_tgt_idx = torch.LongTensor([[TGT.vocab.stoi[w] for w in sent_tgt]]) #stoi – A collections.defaultdict instance mapping token strings to numerical identifiers.
    sent_tgt_idx = Variable(sent_tgt_idx)
    sent_src = SRC.vocab.itos
    sent_src_idx = torch.LongTensor([[SRC.vocab.stoi[w] for w in sent_src]]) #stoi – A collections.defaultdict instance mapping token strings to numerical identifiers.
    sent_src_idx = Variable(sent_src_idx)
    print(sent_src_idx.size())
    # print(sent_src_idx)

    out_emb1 = model.tgt_embed[0].lut
    sent_tgt_idx = out_emb1(sent_tgt_idx.to(device))

    out_emb2 = model.src_embed[0].lut
    sent_src_idx = out_emb2(sent_src_idx.to(device))
    sent_tgt_idx = sent_tgt_idx.squeeze()
    sent_src_idx = sent_src_idx.squeeze()
    sent_idx = torch.cat((sent_tgt_idx[:n_samples], sent_src_idx[:n_samples]),0)
    sent = sent_tgt[:n_samples] + sent_src[:n_samples]

    # Raw embedding
    temp=sent_tgt_idx[:n_samples].cpu().detach().numpy()
    temp1=sent_src_idx[:n_samples].cpu().detach().numpy()

    fig, axs = plt.subplots(3, 3, figsize=(20, 20), dpi= 80, facecolor='w', edgecolor='k')
    axs[0, 0].scatter(temp[:n_samples-1,0], temp[:n_samples-1,1], s=10, alpha=0.5)
    axs[0, 0].scatter(temp1[:n_samples-1,0], temp1[:n_samples-1,1], s=10, alpha=0.5)
    axs[0, 0].set_title('[0,1]')
    axs[0, 1].scatter(temp[:n_samples-1,0], temp[:n_samples-1,2], s=10, alpha=0.5)
    axs[0, 1].scatter(temp1[:n_samples-1,0], temp1[:n_samples-1,2], s=10, alpha=0.5)
    axs[0, 1].set_title('[0,2]')
    axs[0, 2].scatter(temp[:n_samples-1,0], temp[:n_samples-1,3], s=10, alpha=0.5)
    axs[0, 2].scatter(temp1[:n_samples-1,0], temp1[:n_samples-1,3], s=10, alpha=0.5)
    axs[0, 2].set_title('[0,3]')
    axs[1, 0].scatter(temp[:n_samples-1,1], temp[:n_samples-1,2], s=10, alpha=0.5)
    axs[1, 0].scatter(temp1[:n_samples-1,1], temp1[:n_samples-1,2], s=10, alpha=0.5)
    axs[1, 0].set_title('[1,2]')
    axs[1, 1].scatter(temp[:n_samples-1,1], temp[:n_samples-1,3], s=10, alpha=0.5)
    axs[1, 1].scatter(temp1[:n_samples-1,1], temp1[:n_samples-1,3], s=10, alpha=0.5)
    axs[1, 1].set_title('[1,3]')
    axs[1, 2].scatter(temp[:n_samples-1,1], temp[:n_samples-1,4], s=10, alpha=0.5)
    axs[1, 2].scatter(temp1[:n_samples-1,1], temp1[:n_samples-1,4], s=10, alpha=0.5)
    axs[1, 2].set_title('[1,4]')
    axs[2, 0].scatter(temp[:n_samples-1,2], temp[:n_samples-1,3], s=10, alpha=0.5)
    axs[2, 0].scatter(temp1[:n_samples-1,2], temp1[:n_samples-1,3], s=10, alpha=0.5)
    axs[2, 0].set_title('[2,3]')
    axs[2, 1].scatter(temp[:n_samples-1,2], temp[:n_samples-1,4], s=10, alpha=0.5)
    axs[2, 1].scatter(temp1[:n_samples-1,2], temp1[:n_samples-1,4], s=10, alpha=0.5)
    axs[2, 1].set_title('[2,4]')
    axs[2, 2].scatter(temp[:n_samples-1,2], temp[:n_samples-1,5], s=10, alpha=0.5)
    axs[2, 2].scatter(temp1[:n_samples-1,2], temp1[:n_samples-1,5], s=10, alpha=0.5)
    axs[2, 2].set_title('[2,5]')
    # print(temp.shape)
    # ax.scatter(temp[:n_samples-1,0], temp[:n_samples-1,1], s=20)
    # ax.scatter(temp1[:n_samples-1,0], temp[:n_samples-1,1], s=20)
    plt.savefig(os.path.join(directory,'_raw_embedding_' + string_name + '_' + str(epoch) + '.png'))
    # ax.set(title='Raw embedding (1st vs 2nd')
    # np.std(temp, axis=0)
    plt.close('all')


    # n_samples = 1026
    gamma=0
    # print('sent_src_idx[:n_samples].size(): {}'.format(sent_src_idx.size()))
    phate_operator = phate.PHATE(n_components=2, gamma=gamma, n_jobs=-1) #setting just t
    data_phate = phate_operator.fit_transform(sent_idx.cpu().detach().numpy())

    ### Add some landmarks for reference
    idx_tgt_words=[]
    idx_src_words=[]
    tgt_words_land = word_list.iloc[:,1] #['January', 'March', 'April', 'May'] #word_list.ix[:,1]
    src_words_land = word_list.iloc[:,0] #['janvier', 'mars', 'avril', 'mai'] #word_list.ix[:,0]
    # len(tgt_words_land)
    for i in range(len(tgt_words_land)-1):
        for j in range(0,n_samples-1): #scanning TGT
            if tgt_words_land[i]==sent[j]: #if this word is in word_list, find it in french as well
                for k in range(n_samples,2*n_samples-1):
                    if src_words_land[i]==sent[k]:
    #                     print('word[j={},k={}]: {},{}'.format(j,k,sent[j],sent[k]))
                        idx_tgt_words.append(j)
                        idx_src_words.append(k)

    # Plotting the overalpped phate
    # sent[0]
    fig, ax = plt.subplots(figsize=(12, 12), dpi= 80, facecolor='w', edgecolor='k')
    ax.scatter(data_phate[:n_samples-1,0], data_phate[:n_samples-1,1], s=20)
    ax.scatter(data_phate[n_samples:2*n_samples-1,0], data_phate[n_samples:2*n_samples-1,1], s=20, color='g')

    ax.scatter(data_phate[idx_tgt_words,0], data_phate[idx_tgt_words,1], s=20, color='r')
    for i in range(0,len(idx_tgt_words)-1):
        ax.annotate(sent[idx_tgt_words[i]], (data_phate[idx_tgt_words[i],0], data_phate[idx_tgt_words[i],1]), fontsize=15)
        
    ax.scatter(data_phate[idx_src_words,0], data_phate[idx_src_words,1], s=20, color='tab:orange')
    for i in range(0,len(idx_src_words)-1):
        ax.annotate(sent[idx_src_words[i]], (data_phate[idx_src_words[i],0], data_phate[idx_src_words[i],1]), fontsize=15)
    ax.set_title(string_name+'_EN_FR_phate_' + string_name + '_' + str(epoch))
    plt.savefig(os.path.join(directory,'_EN_FR_phate_' + string_name + '_' + str(epoch) + '.png'))
    plt.close('all')


    #Using UMAP
    reducer = umap.UMAP()
    data_phate = reducer.fit_transform(sent_idx.cpu().detach().numpy())
    fig, ax = plt.subplots(figsize=(12, 12), dpi= 80, facecolor='w', edgecolor='k')
    ax.scatter(data_phate[n_samples:2*n_samples-1,0], data_phate[n_samples:2*n_samples-1,1], s=10, color='g')
    ax.scatter(data_phate[:n_samples-1,0], data_phate[:n_samples-1,1], s=10, marker='*')
                
    ax.scatter(data_phate[idx_tgt_words,0], data_phate[idx_tgt_words,1], s=10, color='r')
    for i in range(0,len(idx_tgt_words)-1):
        ax.annotate(sent[idx_tgt_words[i]], (data_phate[idx_tgt_words[i],0], data_phate[idx_tgt_words[i],1]), fontsize=15)
        
    ax.scatter(data_phate[idx_src_words,0], data_phate[idx_src_words,1], s=10, color='tab:orange')
    for i in range(0,len(idx_src_words)-1):
        ax.annotate(sent[idx_src_words[i]], (data_phate[idx_src_words[i],0], data_phate[idx_src_words[i],1]), fontsize=15)
    ax.set_title(string_name+'_EN_FR_umap_landmarks_' + string_name + '_' + str(epoch))
    plt.savefig(os.path.join(directory,'_EN_FR_umap_landmarks_' + string_name + '_' + str(epoch) + '.png'))
    plt.close('all')
    
    fig, ax = plt.subplots(figsize=(12, 12), dpi= 80, facecolor='w', edgecolor='k')
    ax.scatter(data_phate[:n_samples-1,0], data_phate[:n_samples-1,1], s=10, marker='*')
    ax.scatter(data_phate[idx_tgt_words,0], data_phate[idx_tgt_words,1], s=10, color='r')
    for i in range(0,len(idx_tgt_words)-1):
        ax.annotate(sent[idx_tgt_words[i]], (data_phate[idx_tgt_words[i],0], data_phate[idx_tgt_words[i],1]), fontsize=12)
    ax.set_title(string_name+'_EN_umap_landmarks_' + string_name + '_' + str(epoch))
    plt.savefig(os.path.join(directory,'_EN_umap_landmarks_' + string_name + '_' + str(epoch) + '.png'))

    fig, ax = plt.subplots(figsize=(12, 12), dpi= 80, facecolor='w', edgecolor='k')
    ax.scatter(data_phate[n_samples:2*n_samples-1,0], data_phate[n_samples:2*n_samples-1,1], s=10, color='g')
    ax.scatter(data_phate[idx_src_words,0], data_phate[idx_src_words,1], s=10, color='tab:orange')
    for i in range(0,len(idx_src_words)-1):
        ax.annotate(sent[idx_src_words[i]], (data_phate[idx_src_words[i],0], data_phate[idx_src_words[i],1]), fontsize=12)
    ax.set_title(string_name+'_FR_umap_landmarks_' + string_name + '_' + str(epoch))
    plt.savefig(os.path.join(directory,'_FR_umap_landmarks_' + string_name + '_' + str(epoch) + '.png'))
    plt.close('all')

    #Using PCA
    pca = PCA(n_components=2)
    data_phate = pca.fit_transform(sent_idx.cpu().detach().numpy())

    fig, ax = plt.subplots(figsize=(12, 12), dpi= 80, facecolor='w', edgecolor='k')
    ax.scatter(data_phate[n_samples:2*n_samples-1,0], data_phate[n_samples:2*n_samples-1,1], s=10, color='g')
    ax.scatter(data_phate[:n_samples-1,0], data_phate[:n_samples-1,1], s=10, marker='*')
    ax.scatter(data_phate[idx_tgt_words,0], data_phate[idx_tgt_words,1], s=10, color='r')
    for i in range(0,len(idx_tgt_words)-1):
        ax.annotate(sent[idx_tgt_words[i]], (data_phate[idx_tgt_words[i],0], data_phate[idx_tgt_words[i],1]), fontsize=15)
        
    ax.scatter(data_phate[idx_src_words,0], data_phate[idx_src_words,1], s=10, color='tab:orange')
    for i in range(0,len(idx_src_words)-1):
        ax.annotate(sent[idx_src_words[i]], (data_phate[idx_src_words[i],0], data_phate[idx_src_words[i],1]), fontsize=15)
    ax.set_title(string_name+'_EN_FR_pca_landmarks_' + string_name + '_' + str(epoch))
    plt.savefig(os.path.join(directory,'_EN_FR_pca_landmarks_' + string_name + '_' + str(epoch) + '.png'))
    plt.close('all')

    # Plotting the overalpped phate
    # sent[0]
    fig, ax = plt.subplots(figsize=(15, 16), dpi= 80, facecolor='w', edgecolor='k')
    ax.scatter(data_phate[:n_samples-1,0], data_phate[:n_samples-1,1], s=10, marker='*')
    len(idx_src_words)
    len(idx_src_words)
    ax.scatter(data_phate[idx_tgt_words,0], data_phate[idx_tgt_words,1], s=10, color='r')
    for i in range(0,len(idx_tgt_words)-1):
        ax.annotate(sent[idx_tgt_words[i]], (data_phate[idx_tgt_words[i],0], data_phate[idx_tgt_words[i],1]), fontsize=12)
    ax.set_title(string_name+'_EN_pca_landmarks_' + string_name + '_' + str(epoch))
    plt.savefig(os.path.join(directory,'_EN_pca_landmarks_' + string_name + '_' + str(epoch) + '.png'))
    plt.close('all')

    fig, ax = plt.subplots(figsize=(15, 16), dpi= 80, facecolor='w', edgecolor='k')
    ax.scatter(data_phate[n_samples:2*n_samples-1,0], data_phate[n_samples:2*n_samples-1,1], s=10, color='g')
    ax.scatter(data_phate[idx_src_words,0], data_phate[idx_src_words,1], s=10, color='tab:orange')
    for i in range(0,len(idx_src_words)-1):
        ax.annotate(sent[idx_src_words[i]], (data_phate[idx_src_words[i],0], data_phate[idx_src_words[i],1]), fontsize=12)
    ax.set_title(string_name+'_FR_pca_landmarks_' + string_name + '_' + str(epoch))
    plt.savefig(os.path.join(directory,'_FR_pca_landmarks_' + string_name + '_' + str(epoch) + '.png'))
    plt.close('all')




# def train_generator(optimizer, french_data, english_data):

#     optimizer.zero_grad()
    
#     d_french = discriminator(french_data)
#     d_english = discriminator(english_data)
    
#     loss = torch.mean((d_french - d_english)**2) # minimize difference between french and english
    
#     loss.backward()
#     optimizer.step()

#     return loss


class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many 
    other models.
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab))
    """
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
        
    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        # out = model.forward(batch.src.to(device), batch.trg.to(device), batch.src_mask.to(device), batch.trg_mask.to(device))
        return self.decode(self.encode(src, src_mask), src_mask,
                            tgt, tgt_mask)
    
    def encode(self, src, src_mask):
        # print('(in encoder) src.size: {}'.format(src.size()))
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

class Generator(nn.Module):
    "Define standard linear + softmax generation step."
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

class Encoder(nn.Module):
    "Core encoder is a stack of N layers"
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        # print('(in encoder) x.size: {}'.format(x.size()))
        return self.norm(x)

class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

class Decoder(nn.Module):
    "Generic N layer decoder with masking."
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        # print('(in decoder - before layer) x.size: {}'.format(x.size()))
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        # print('(in decoder) x.size: {}'.format(x.size()))
        return self.norm(x)

class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src-attn, and feed forward (defined below)"
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
 
    def forward(self, x, memory, src_mask, tgt_mask):
        "Follow Figure 1 (right) for connections."
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0

def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0., max_len).unsqueeze(1)
        div_term = 1 / (10000 ** (torch.arange(0., d_model, 2) / d_model))
        # div_term = torch.exp(torch.arange(0., d_model, 2) *
                             # -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)


def make_model(src_vocab, tgt_vocab, N=6, 
               d_model=512, d_ff=2048, h=8, dropout=0.1):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab))


    # This was important from their code. 
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model

class Batch:
    "Object for holding a batch of data with mask during training."
    def __init__(self, src, trg=None, pad=0):
        self.src = src
        self.src_mask = (src != pad).unsqueeze(-2)
        if trg is not None:
            self.trg = trg[:, :-1]
            self.trg_y = trg[:, 1:]
            self.trg_mask = \
                self.make_std_mask(self.trg, pad)
            self.ntokens = (self.trg_y != pad).data.sum()
    
    @staticmethod
    def make_std_mask(tgt, pad): 
        "Create a mask to hide padding and future words."
        tgt_mask = (tgt != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & Variable(
            subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data))
        return tgt_mask

def run_epoch(data_iter, model, loss_compute, device, SRC, TGT, epoch, n_epochs):
    "Standard Training and Logging Function"
    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0
    start = time.time()
    total_tokens = 0
    total_loss = 0
    total_loss_generator=0
    total_loss_transformer=0
    total_loss_MMD = 0
    total_loss_justTransf = 0
    tokens = 0
    for i, batch in enumerate(data_iter):
        # batch = batch.cuda()
        # words = word_list.sample(frac=0.05) #Maybe n=100 words as landmarks is not enough... Try frac=0.5
        
        # #using landmarks
        # src_words = words.ix[:,0] 
        # tgt_words = words.ix[:,1]
        # n_samples_mmd = 1000  #Number of samples
        # word_list = [sample(SRC.vocab.itos,n_samples_mmd), sample(TGT.vocab.itos,n_samples_mmd)]

        # src_words = sample(SRC.vocab.itos,n_samples_mmd)
        # tgt_words = sample(TGT.vocab.itos,n_samples_mmd)

        # src_words = torch.LongTensor([[SRC.vocab.stoi[w] for w in src_words]]) #stoi – A collections.defaultdict instance mapping token strings to numerical identifiers.
        # src_words = Variable(src_words)
        # tgt_words = torch.LongTensor([[TGT.vocab.stoi[w] for w in tgt_words]]) #stoi – A collections.defaultdict instance mapping token strings to numerical identifiers.
        # tgt_words = Variable(tgt_words)
        # print("tgt_words.size(): {}".format(tgt_words.size()))
        # print("tgt_words: {}".format(tgt_words))
        
        # print('batch.trg.data: {}'.format(batch.trg.data))
        # tgt_words = torch.unique(batch.trg.data, sorted=False) #Extracting unique words from the batch 
        # src_words = torch.unique(batch.src.data, sorted=False) 

        # Using GANs#
        # tgt_words = torch.reshape(batch.trg.data, (-1,))
        # tgt_words = tgt_words[tgt_words>3]
        # src_words = torch.reshape(batch.src.data, (-1,))
        # src_words = src_words[src_words>3]
        # min_words = min(tgt_words.size(0), src_words.size(0))
        # tgt_words = tgt_words[:min_words]
        # src_words = src_words[:min_words]
        # \Using GANs#


        # print("tgt_words.size(): {}, src_words.size(): {}, min_words: {}".format(tgt_words.size(), src_words.size(), min_words))
        # print('tgt_words: {}'.format(tgt_words))
        # print('batch.src.data.size(): {}'.format(batch.src.data.size()))
        # break
        # print('batch.src: {}'.format(batch.src))
        # print('Number of sentences: {}'.format(batch.trg.size(0))) #Number of sentences in this batch (batch.trg.size(0) and batch.src.size(0) are equal)
        # print('batch.src.size(): {}'.format(batch.src.size()))
        # if i==1:
        #     print("Target:", end="\t")
        #     for j in range(1, batch.trg.size(1)): #Maximum sentence length in this batch
        #         sym = TGT.vocab.itos[batch.trg.data[0, j]] #Translating indexes to words of the first english sentence in this batch.
        #         if sym == "</s>": break
        #         # print(sym, end =" ")
        #         print("(j={}) {}".format(batch.trg.data[0, j],sym))
        #     print()
        # if i==1:
        #     print("Target:", end="\t")
        #     for j in range(1, tgt_words.size(0)): #Maximum sentence length in this batch
        #         sym = TGT.vocab.itos[tgt_words[j]] #Translating indexes to words of the first english sentence in this batch.
        #         if sym == "</s>": break
        #         # print(sym, end =" ")
        #         print("(j={}) {}".format(tgt_words[j],sym))
        #     print()
        #     print("Source:", end="\t")
        #     for j in range(1, src_words.size(0)): #Maximum sentence length in this batch
        #         sym = SRC.vocab.itos[src_words[j]] #Translating indexes to words of the first english sentence in this batch.
        #         if sym == "</s>": break
        #         # print(sym, end =" ")
        #         print("(j={}) {}".format(src_words[j],sym))
        #     print()
        # print("i={}\n".format(i))
        # if i==2:
        #     break

        # Select src_words and tgt_words from word_list (500 words / 50 for every epoch)
        out = model.forward(batch.src.to(device), batch.trg.to(device), batch.src_mask.to(device), batch.trg_mask.to(device))
        # print('out.size(): {}'.format(out.size()))
#         try:
        #### Using MMD ####
        # loss, loss_MMD, loss_trans = loss_compute(out, batch.trg_y.to(device), batch.ntokens.to(device), src_words, tgt_words) 
        #### Using MMD ####

        # #### Using GANs ####
        # #Compute embeddings
        # embed_src = model.src_embed[0].lut(batch.src.to(device)).cpu().detach()
        # # embed_src = embed_src.to(device)
        # embed_tgt = model.tgt_embed[0].lut(batch.trg.to(device)).cpu().detach()
        # # embed_tgt = embed_tgt.to(device)
        # #Compute loss for the discriminator
        # loss_discriminator, d_pred_french, d_pred_english = train_discriminator(d_optimizer, batch.src.to(device), batch.trg.to(device))
        # #Compute transformer loss (full step, thus including update of the embedding) and generator loss
        # # loss, loss_generator, loss_transformer = loss_compute(out, batch.trg_y.to(device), batch.ntokens.to(device), src_words, tgt_words) 
        # #### \Using GANs ####

        # self, x, y, norm, src_words, tgt_words
        # loss.item(), loss_generator.item(),  loss_transformer.item()
        if EUCLDN:
            loss, loss_generator, loss_transformer = loss_compute(out, batch.trg_y.to(device), batch.ntokens.to(device), batch.src.to(device), batch.trg.to(device)) 
            total_loss_generator += loss_generator
        elif MMD:
            loss, loss_MMD, loss_transformer = loss_compute(out, batch.trg_y.to(device), batch.ntokens.to(device), batch.src.to(device), batch.trg.to(device)) 
            total_loss_MMD += loss_MMD
        else:
            loss, loss_justTransf, loss_transformer = loss_compute(out, batch.trg_y.to(device), batch.ntokens.to(device), batch.src.to(device), batch.trg.to(device)) 
            total_loss_justTransf += loss_justTransf
        # loss, loss_land, loss_trans = loss_compute(out, batch.trg_y.to(device), batch.ntokens.to(device), batch.src.to(device), batch.trg.to(device))
#         except:
#             print('Unexpected error: {}'.format(sys.exc_info()[0]))
#             del batch
        # print('Loss: {}'.format(str(loss)))
        total_loss += loss
        total_loss_transformer+= loss_transformer
        total_tokens += batch.ntokens.item()
        tokens += batch.ntokens.item()
        

        if i % 100 == 0:
            elapsed = time.time() - start
            # print(batch.ntokens.item())
            # print('Loss: {}'.format(loss))
            # print('Epoch Step: {} Loss: {}'.format(i, loss / batch.ntokens.item()))
            if EUCLDN:
                print('Epoch Step: {} Loss: {:.4f} (loss_gener: {:.2e}, loss_trans: {:.4f}) Tokens per Sec: {:.4f}'.format(i, loss, loss_generator, loss_transformer, tokens / elapsed))
            elif MMD:
                print('Epoch Step: {} Loss: {:.4f} (loss_MMD: {:.4f}, loss_trans: {:.4f}) Tokens per Sec: {:.4f}'.format(i, loss, loss_MMD, loss_transformer, tokens / elapsed))
            else:
                print('Epoch Step: {} Loss: {:.4f} (loss_trans: {:.4f}) Tokens per Sec: {:.4f}'.format(i, loss, loss_transformer, tokens / elapsed))
            start = time.time()
            tokens = 0

        # logger.log(loss, loss_transformer, loss_MMD, loss_generator, epoch, i)
        # Display Progress every few batches
        # if i % 100 == 0: 
            # test_images = vectors_to_images(generator(test_noise))
            # test_images = test_images.data
            # logger.log_images(
            #     test_images, num_test_samples, 
            #     epoch, n_batch, num_batches
            # );
            # Display status Logs
            # logger.display_status(
            #     epoch, n_epochs, i,
            #     loss_transformer, loss_discriminator, loss_generator, d_pred_french, d_pred_english
            # )
    # loss_train, loss_total_train, loss_generator_train, loss_discriminator_train, loss_transformer_train
    if EUCLDN:
        return total_loss / total_tokens, loss, total_loss_generator / total_tokens, total_loss_transformer / total_tokens
    elif MMD:
        return total_loss / total_tokens, loss, total_loss_MMD/total_tokens, total_loss_transformer / total_tokens
    else:
        return total_loss / total_tokens, loss, total_loss_justTransf/total_tokens, total_loss_transformer / total_tokens

global max_src_in_batch, max_tgt_in_batch
def batch_size_fn(new, count, sofar):
    "Keep augmenting batch and calculate total number of tokens + padding."
    global max_src_in_batch, max_tgt_in_batch
    if count == 1:
        max_src_in_batch = 0
        max_tgt_in_batch = 0
    max_src_in_batch = max(max_src_in_batch,  len(new.src))
    max_tgt_in_batch = max(max_tgt_in_batch,  len(new.trg) + 2)
    src_elements = count * max_src_in_batch
    tgt_elements = count * max_tgt_in_batch
    return max(src_elements, tgt_elements)

class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup ** (-1.5)))
        
def get_std_opt(model):
    return NoamOpt(model.src_embed[0].d_model, 2, 4000,
            torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))

class LabelSmoothing(nn.Module):
    "Implement label smoothing."
    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(size_average=False)
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None
        
    def forward(self, x, target):
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        return self.criterion(x, Variable(true_dist, requires_grad=False))

def loss(x):
    d = x + 3 * 1
    predict = torch.FloatTensor([[0, x / d, 1 / d, 1 / d, 1 / d],
                                 ])
    #print(predict)
    return crit(Variable(predict.log()),
                 Variable(torch.LongTensor([1]))).item()
    # return crit(Variable(predict.log()),
    #              Variable(torch.LongTensor([1]))).data[0]

#Finally an example
def data_gen(V, batch, nbatches):
    "Generate random data for a src-tgt copy task."
    for i in range(nbatches):
        data = torch.from_numpy(np.random.randint(1, V, size=(batch, 10)))
        data[:, 0] = 1
        src = Variable(data, requires_grad=False)
        tgt = Variable(data, requires_grad=False)
        yield Batch(src, tgt, 0)

class SimpleLossCompute:
    "A simple loss compute and train function."
    # def __init__(self, generator, criterion, src_embed, tgt_embed, SRC, TGT, mmd, opt=None):
    def __init__(self, generator, criterion, src_embed, tgt_embed, SRC, TGT, mmd, opt=None):
        self.generator = generator
        self.criterion = criterion
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.SRC = SRC
        self.TGT = TGT
        self.mmd=mmd
        # self.discriminator = discriminator
        # self.g_optimizer = g_optimizer
        # self.loss_GAN = loss_GAN
        # self.english_data_target = english_data_target
        # self.mmd=mmd
        self.opt = opt
        
    # def __call__(self, x, y, norm, src_words, tgt_words): #using landmarks
    def __call__(self, x, y, norm, src_words, tgt_words):
        # print('x.size(): {}'.format(x.size()))
        x = self.generator(x)
        # print('(post genertator) x.size(): {}'.format(x.size()))
        # print('x.contiguous().view(-1, x.size(-1)).size(): {}'.format(x.contiguous().view(-1, x.size(-1)).size()))
        # print('y.contiguous().view(-1).size(): {}'.format(y.contiguous().view(-1).size()))
        loss_transformer = self.criterion(x.contiguous().view(-1, x.size(-1)), y.contiguous().view(-1))*weight_loss_transformer / norm
        sigma_list = np.logspace(-3, 2, 10)
        # Subset of words in english/french
#         print('self.src_embed: {}'.format(self.src_embed))
#         print('self.tgt_embed: {}'.format(self.tgt_embed))
#         src_words.columns = ['words']
#         print('src_words: \n{}'.format(src_words))

#         #### using landmarks
#         src_words = src_words.values.tolist()
#         tgt_words = tgt_words.values.tolist()
# #         print('src_words: \n{}'.format(src_words))
        
        # src_words = torch.LongTensor([[self.SRC.vocab.stoi[w] for w in src_words]]) #stoi – A collections.defaultdict instance mapping token strings to numerical identifiers.
        # src_words = Variable(src_words)
        
        # tgt_words = torch.LongTensor([[self.TGT.vocab.stoi[w] for w in tgt_words]]) #stoi – A collections.defaultdict instance mapping token strings to numerical identifiers.
        # tgt_words = Variable(tgt_words)

        # loss_landmarks = 100000*torch.mean((embed_src - embed_tgt)**2)
        # loss_landmarks = 100000*nn.MSELoss()(embed_src, embed_tgt) 
#         #### \using landmarks
          
        # # #### using MMD #######
        # # print('src_words: \n{}'.format(src_words['french'].values))
        # # src_words = torch.stack(src_words)
        # # # print('src_words.size(): {}'.format(src_words.size()))
        # # # print('tgt_words.size(): {}'.format(tgt_words.size()))
        # embed_src = self.src_embed(src_words.to(device)).cpu()
        # embed_tgt = self.tgt_embed(tgt_words.to(device)).cpu()
        # # # print('embed_src.size(): {}'.format(embed_src.size()))
        # # # print('embed_tgt.size(): {}'.format(embed_tgt.size()))
        # # loss_MMD = self.mmd.mix_rbf_mmd2(embed_src.squeeze(), embed_tgt.squeeze(), sigma_list=sigma_list)
        # # loss_MMD = weigth_MMD*loss_MMD
        # # #### \using MMD #######

        # Using MMD (v2)
        # print("tgt_words.size(): {}, src_words.size(): {}".format(tgt_words.size(), src_words.size()))
        

        # batch_MMD_loss = []
        # for j in range(0, src_words.size(0)):
        #     # print("src_words.size(): {}\n tgt_words.size(): {}".format(src_words.size(), tgt_words.size()))
        #     src_words2 = src_words[j,src_words[j,:]!=1] #1=<blank>
        #     tgt_words2 = tgt_words[j,tgt_words[j,:]>3] #1=<blank>, 2=<s>, 3=</s>
        #     # print("src_words2_orig: {}\n tgt_words2_orig: {}".format(src_words2, tgt_words2))
        #     # print("src_words2.size(): {}\n tgt_words2.size(): {}".format(src_words2.size(), tgt_words2.size()))
        #     min_words = min(tgt_words2.size(0), src_words2.size(0))
        #     src_words2 = src_words2[torch.multinomial(src_words2.float(),min_words)]
        #     tgt_words2 = tgt_words2[torch.multinomial(tgt_words2.float(),min_words)]
        #     # tgt_words2 = tgt_words2[:,:min_words]
        #     # src_words2 = src_words2[:,:min_words]
        #     # print("Target:", end="\t")
        #     # for k in range(0, tgt_words2.size(0)): #Maximum sentence length in this batch
        #     #     sym = TGT.vocab.itos[tgt_words2[k]] #Translating indexes to words of the first english sentence in this batch.
        #     #     if sym == "</s>": break
        #     #     # print(sym, end =" ")
        #     #     print("(j={}) {}".format(tgt_words2[k],sym))
        #     # print()
        #     # print("Source:", end="\t")
        #     # for k in range(0, src_words2.size(0)): #Maximum sentence length in this batch
        #     #     sym = SRC.vocab.itos[src_words2[k]] #Translating indexes to words of the first english sentence in this batch.
        #     #     if sym == "</s>": break
        #     #     # print(sym, end =" ")
        #     #     print("(j={}) {}".format(src_words2[k],sym))
        #     # print()
        #     # print("src_words.size(): {}, tgt_words.size(): {}".format(src_words.size(), tgt_words.size()))
        #     # print("src_words2: {}\n tgt_words2: {}".format(src_words2, tgt_words2))
        #     embed_src = model.src_embed[0].lut(src_words2.to(device))
        #     embed_tgt = model.tgt_embed[0].lut(tgt_words2.to(device))
        #     # print("tgt_words2.size(): {}, src_words2.size(): {}, embed_src.size: {}".format(tgt_words2.size(), src_words2.size(), embed_src.size()))
        #     batch_MMD_loss.append(self.mmd.mix_rbf_mmd2(embed_src, embed_tgt, sigma_list=sigma_list))
        #     # break
        # loss_MMD =  torch.mean(torch.stack(batch_MMD_loss)).to(device)*weight_loss_MMD
        # # print("loss_MMD: {}".format(loss_MMD))
        # # print("loss_transformer: {}".format(loss_transformer))
        # loss = weight_loss_transformer*loss_transformer + loss_MMD

        # # \Using MMD (v2)

        # # Using MSE for sentence pairs#
        
        # batch_MSE_loss = []
        # for j in range(1, src_words.size(0)):
        #     # print("src_words: {}\n, tgt_words: {}".format(src_words, tgt_words))
        #     src_words2 = src_words[j,:]
        #     tgt_words2 = tgt_words[j,:]
        #     # print("src_words.size(): {}, tgt_words.size(): {}".format(src_words.size(), tgt_words.size()))
        #     # print("src_words: {}\n tgt_words: {}".format(src_words, tgt_words))
        #     embed_src = torch.mean(model.src_embed[0].lut(src_words2.to(device)).cpu().detach())
        #     embed_tgt = torch.mean(model.tgt_embed[0].lut(tgt_words2.to(device)).cpu())
        #     # print("tgt_words.size(): {}, src_words.size(): {}, embed_src: {}".format(tgt_words.size(), src_words.size(), embed_src))
        #     batch_MSE_loss.append(nn.MSELoss()(embed_src, embed_tgt))
        # loss_MSE =  torch.mean(torch.stack(batch_MSE_loss)).to(device)*weight_loss_MSE
        # # print("loss_MSE: {}".format(loss_MSE))
        # # print("loss_transformer: {}".format(loss_transformer))
        
        # loss = weight_loss_transformer*loss_transformer + loss_MSE
        # # \Using MSE for sentence pairs#        
        
        #### Using GANs ####
        # loss_generator = self.train_generator_gan(self.g_optimizer, embed_src, embed_tgt)
        if EUCLDN:
            batch_GAN_loss = []
            # try:
            for j in range(1, src_words.size(0)-1):

                # 1st pair of sentences 
                src_words_1 = src_words[j,src_words[j,:]!=1] #1=<blank>
                tgt_words_1 = tgt_words[j,tgt_words[j,:]>3] #1=<blank>, 2=<s>, 3=</s>
                # src_words_2 = src_words[j+1,src_words[j+1,:]!=1] #1=<blank>
                # tgt_words_2 = tgt_words[j+1,tgt_words[j+1,:]>3] #1=<blank>, 2=<s>, 3=</s>
                # print("src_words_1: {}".format(src_words_1))
                # print("src_words_1.size(): {}".format(src_words_1.size()))
                
                # min_words = min(tgt_words_1.size(0), src_words_1.size(0),tgt_words_2.size(0), src_words_2.size(0))
                # src_words_1 = src_words_1[torch.multinomial(src_words_1.float(),min_words)]
                # tgt_words_1 = tgt_words_1[torch.multinomial(tgt_words_1.float(),min_words)]
                # embed_src_1 = model.src_embed[0].lut(src_words_1.to(device)).cpu()
                # print("embed_src_1: {}".format(embed_src_1))
                # print("embed_src_1.size(): {}".format(embed_src_1.size()))
                embed_src_1 = torch.mean(model.src_embed[0].lut(src_words_1.to(device)).cpu(),0)
                embed_tgt_1 = torch.mean(model.tgt_embed[0].lut(tgt_words_1.to(device)).cpu(),0)
                # print("embed_src_1: {}".format(embed_src_1))
                # print("embed_src_1.size(): {}".format(embed_src_1.size()))

                # 2nd pair of sentences 
                # src_words_2 = src_words_2[torch.multinomial(src_words_2.float(),min_words)]
                # tgt_words_2 = tgt_words_2[torch.multinomial(tgt_words_2.float(),min_words)]
                # embed_src_2 = torch.mean(model.src_embed[0].lut(src_words_2.to(device)).cpu(),0)
                # embed_tgt_2 = torch.mean(model.tgt_embed[0].lut(tgt_words_2.to(device)).cpu(),0)
            # print("src_words: {}".format(src_words))
            # print("src_words.size(): {}".format(src_words.size()))
            # embed_src = model.src_embed[0].lut(src_words.to(device)).cpu()
            # print("embed_src: {}".format(embed_src))
            # print("embed_src.size(): {}".format(embed_src.size()))
            # embed_src = torch.mean(model.src_embed[0].lut(src_words.to(device)).cpu(),0)
            # embed_tgt = torch.mean(model.tgt_embed[0].lut(tgt_words.to(device)).cpu(),0)
            # print("embed_src: {}".format(embed_src))
            # print("embed_src.size(): {}".format(embed_src.size()))
                # prediction_french = self.discriminator(embed_src)
                # prediction_english = self.discriminator(embed_tgt)
                # prediction_french = embed_src_1
                # prediction_english = embed_tgt_1
                # loss_generator = torch.mean((d_french - d_english)**2) # minimize difference between french and english
                # loss_generator = self.loss_GAN(prediction_english, french_data_target(prediction_english.squeeze().size(0)))
                # print("torch.norm(prediction_english - prediction_french,p=2): {}".format(torch.norm(prediction_english - prediction_french,p=2)))
                # loss_generator = 2*torch.norm(prediction_english - prediction_french,p=2) - torch.norm(prediction_english,p=2) - torch.norm(prediction_french,p=2)
                # loss_generator = torch.norm(embed_tgt_1 - embed_src_1,p=2) + torch.norm(embed_tgt_2 - embed_src_2,p=2) - torch.norm(embed_tgt_1 - embed_src_2,p=2) - torch.norm(embed_src_1 - embed_tgt_2,p=2)
                loss_generator = -torch.dot(embed_tgt_1, embed_src_1)
                # print("embed_src_1.size(): {} , embed_tgt_1.size(): {}, loss_generator: {}".format(embed_src_1.size(), embed_tgt_1.size(),loss_generator))
                # loss_generator = (embed_tgt_1 - embed_src_1)**2 + (embed_tgt_2 - embed_src_2)**2 - (embed_tgt_1 - embed_tgt_2)**2 - (embed_src_1 - embed_src_2)**2
                # loss_generator = 2*(torch.mean(prediction_french) - torch.mean(prediction_english))**2 - torch.mean(prediction_english)**2 - torch.mean(prediction_french)**2
                # print("prediction_english: {}".format(prediction_english))
                # print("prediction_french: {}".format(prediction_french))
                # loss_generator = self.loss_GAN(prediction_english, prediction_french.detach())
                # loss_generator.backward()
                batch_GAN_loss.append(loss_generator)
                # print("len(batch_GAN_loss): {}".format(len(batch_GAN_loss)))
            loss_generator = torch.mean(torch.stack(batch_GAN_loss)).to(device)*weight_loss_generator
            loss = loss_transformer + loss_generator
        elif MMD:
            batch_MMD_loss = []
            for j in range(0, src_words.size(0)):
                # print("src_words.size(): {}\n tgt_words.size(): {}".format(src_words.size(), tgt_words.size()))
                src_words2 = src_words[j,src_words[j,:]!=1] #1=<blank>
                tgt_words2 = tgt_words[j,tgt_words[j,:]>3] #1=<blank>, 2=<s>, 3=</s>
                # print("src_words2_orig: {}\n tgt_words2_orig: {}".format(src_words2, tgt_words2))
                # print("src_words2.size(): {}\n tgt_words2.size(): {}".format(src_words2.size(), tgt_words2.size()))
                min_words = min(tgt_words2.size(0), src_words2.size(0))
                src_words2 = src_words2[torch.multinomial(src_words2.float(),min_words)]
                tgt_words2 = tgt_words2[torch.multinomial(tgt_words2.float(),min_words)]
                # tgt_words2 = tgt_words2[:,:min_words]
                # src_words2 = src_words2[:,:min_words]
                # print("Target:", end="\t")
                # for k in range(0, tgt_words2.size(0)): #Maximum sentence length in this batch
                #     sym = TGT.vocab.itos[tgt_words2[k]] #Translating indexes to words of the first english sentence in this batch.
                #     if sym == "</s>": break
                #     # print(sym, end =" ")
                #     print("(j={}) {}".format(tgt_words2[k],sym))
                # print()
                # print("Source:", end="\t")
                # for k in range(0, src_words2.size(0)): #Maximum sentence length in this batch
                #     sym = SRC.vocab.itos[src_words2[k]] #Translating indexes to words of the first english sentence in this batch.
                #     if sym == "</s>": break
                #     # print(sym, end =" ")
                #     print("(j={}) {}".format(src_words2[k],sym))
                # print()
                # print("src_words.size(): {}, tgt_words.size(): {}".format(src_words.size(), tgt_words.size()))
                # print("src_words2: {}\n tgt_words2: {}".format(src_words2, tgt_words2))
                embed_src = model.src_embed[0].lut(src_words2.to(device))
                embed_tgt = model.tgt_embed[0].lut(tgt_words2.to(device))
                # print("tgt_words2.size(): {}, src_words2.size(): {}, embed_src.size: {}".format(tgt_words2.size(), src_words2.size(), embed_src.size()))
                batch_MMD_loss.append(self.mmd.mix_rbf_mmd2(embed_src, embed_tgt, sigma_list=sigma_list))
                # break
            loss_MMD =  torch.mean(torch.stack(batch_MMD_loss)).to(device)*weight_loss_MMD
            loss = loss_transformer + loss_MMD
        else:
            batch_justTransf_loss = []
            for j in range(0, src_words.size(0)):
                # print("src_words.size(): {}\n tgt_words.size(): {}".format(src_words.size(), tgt_words.size()))
                src_words2 = src_words[j,src_words[j,:]!=1] #1=<blank>
                tgt_words2 = tgt_words[j,tgt_words[j,:]>3] #1=<blank>, 2=<s>, 3=</s>
                # print("src_words2_orig: {}\n tgt_words2_orig: {}".format(src_words2, tgt_words2))
                # print("src_words2.size(): {}\n tgt_words2.size(): {}".format(src_words2.size(), tgt_words2.size()))
                min_words = min(tgt_words2.size(0), src_words2.size(0))
                src_words2 = src_words2[torch.multinomial(src_words2.float(),min_words)]
                tgt_words2 = tgt_words2[torch.multinomial(tgt_words2.float(),min_words)]
                
                embed_src = model.src_embed[0].lut(src_words2.to(device))
                embed_tgt = model.tgt_embed[0].lut(tgt_words2.to(device))
                # print("tgt_words2.size(): {}, src_words2.size(): {}, embed_src.size: {}".format(tgt_words2.size(), src_words2.size(), embed_src.size()))
                batch_justTransf_loss.append(self.mmd.mix_rbf_mmd2(embed_src, embed_tgt, sigma_list=sigma_list))
                # break
            loss_justTransf =  torch.mean(torch.stack(batch_justTransf_loss)).to(device)*weight_loss_MMD

            loss = loss_transformer
        # except: # in case the batch_GAN_loss comes empty
        #     print('error with batch_GAN_loss')
        #     try:
        #         print("loss_generator: {}".format(loss_generator))
        #     except:
        #         print("loss_generator=0")
        #         loss_generator=torch.cuda.FloatTensor([0])
        
        #### \Using GANs ####

        # print('loss_landmarks: {}'.format(loss_landmarks))
        # print("loss.item(): {}, loss_MMD.item(): {},  loss_transformer.item(): {}".format(loss.item(), loss_MMD.item(),  loss_transformer.item()))
        loss.backward()

        # print('Norm: {}'.format(str(norm)))
        if self.opt is not None:
            self.opt.step()
            self.opt.optimizer.zero_grad()
            # self.g_optimizer.step()
            # self.g_optimizer.zero_grad()
        # print('Loss: {}'.format(str(loss.detach().numpy())))
        # print('Loss: {}'.format(str(loss.item())))
        # print('Loss*norm: {}'.format(str(loss.item() * norm.item())))
        # return loss.data[0] * norm #Original function
        # loss = loss*norm
        if EUCLDN:
            return loss.item(), loss_generator.item(),  loss_transformer.item()
        elif MMD:
            return loss.item(), loss_MMD.item(),  loss_transformer.item()
        else:
            return loss.item(),  loss_transformer.item(), loss_transformer.item()

def greedy_decode(model, src, src_mask, max_len, start_symbol):
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len-1):
        out = model.decode(memory, src_mask, 
                           Variable(ys), 
                           Variable(subsequent_mask(ys.size(1))
                                    .type_as(src.data)))
#         print('out_dim: {}'.format(len(out)))
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.data[0]
        ys = torch.cat([ys, 
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
    return ys

class MyIterator(data.Iterator):
    def create_batches(self):
        if self.train:
            def pool(d, random_shuffler):
                for p in data.batch(d, self.batch_size * 100):
                    p_batch = data.batch(
                        sorted(p, key=self.sort_key),
                        self.batch_size, self.batch_size_fn)
                    for b in random_shuffler(list(p_batch)):
                        yield b
            self.batches = pool(self.data(), self.random_shuffler)
            
        else:
            self.batches = []
            for b in data.batch(self.data(), self.batch_size,
                                          self.batch_size_fn):
                self.batches.append(sorted(b, key=self.sort_key))

def rebatch(pad_idx, batch):
    "Fix order in torchtext to match ours"
    src, trg = batch.src.transpose(0, 1), batch.trg.transpose(0, 1)
    return Batch(src, trg, pad_idx)

print('\nDone defining functions...')


Done defining functions...


In [3]:
word_list = pd.read_csv('/gpfs/ysm/home/ahf38/project/word_embed_align/word_list.csv')
word_list.columns = ['french', 'english']
# print(word_list.shape)
# print(word_list.head(500))

In [11]:
# europarl_en = open('/home/antonio/Documents/MovingMNIST/europarl-v7.fr-en.en', encoding='utf-8').read().split('\n')
# europarl_fr = open('/home/antonio/Documents/MovingMNIST/europarl-v7.fr-en.fr', encoding='utf-8').read().split('\n')
global n_samples_mmd, weigth_MMD, weight_loss_generator
number_of_lines = 1000000 # number of sentences: 100k
# n_samples_mmd = 200  #Number of samples for MMD
BATCH_SIZE = 2500 # Number of words per batch: 7k
weight_loss_transformer = 1
weight_loss_generator = 100
path_to_save_models = '/gpfs/ysm/home/ahf38/project/word_embed_align/models'
path_to_save_plots = '/gpfs/ysm/home/ahf38/project/word_embed_align/plots'
string_name = 'transformer_GANs_withMMD_1MSent'
path_save = path_to_save_plots
# weight_loss_discriminator = 0.5
weight_loss_MMD = 10
MMD = True
EUCLDN = False
n_epochs=10000
patience=100
# generator = GeneratorNet()


europarl_en = islice(open('/gpfs/ysm/home/ahf38/project/word_embed_align/europarl-v7.fr-en.en', encoding='utf-8').read().split('\n'), number_of_lines)
europarl_fr = islice(open('/gpfs/ysm/home/ahf38/project/word_embed_align/europarl-v7.fr-en.fr', encoding='utf-8').read().split('\n'), number_of_lines)
# word_list = pd.read_csv('/home/antonio/Documents/MovingMNIST/word_list.csv')
# word_list.columns = ['french', 'english']
# for current_line in europarl_en:
#     print (current_line)
# print(word_list.shape)
# print(word_list.head(500))

######## Real world example #######
if True:
    import spacy #  Spacy is a library that has been specifically built to take sentences in various languages and split them into different tokens 
    import pandas as pd
    from torchtext.data import Field, BucketIterator, TabularDataset
    from sklearn.model_selection import train_test_split

    spacy_fr = spacy.load('fr')
    spacy_en = spacy.load('en')

    def tokenize_fr(text):
        return [tok.text for tok in spacy_fr.tokenizer(text)]

    def tokenize_en(text):
        return [tok.text for tok in spacy_en.tokenizer(text)]

    BOS_WORD = '<s>'
    EOS_WORD = '</s>'
    BLANK_WORD = "<blank>"
    # SRC -> French; TGT -> English
    SRC = data.Field(tokenize=tokenize_fr, pad_token=BLANK_WORD)
    TGT = data.Field(tokenize=tokenize_en, init_token = BOS_WORD, eos_token = EOS_WORD, pad_token=BLANK_WORD)

    # Save the data to csv for inspection
    raw_data = {'English' : [line for line in europarl_en], 'French': [line for line in europarl_fr]}
    df = pd.DataFrame(raw_data, columns=["English", "French"])
    # remove very long sentences and sentences where translations are 
    # not of roughly equal length
    df['eng_len'] = df['English'].str.count(' ')
    df['fr_len'] = df['French'].str.count(' ')
    df = df.query('fr_len < 80 & eng_len < 80')
    df = df.query('fr_len < eng_len * 1.5 & fr_len * 1.5 > eng_len')
    # df = df[df['English'].str.contains('football')]
    number_of_lines = df.shape
    # print(df['English'].head(500))
    if EUCLDN:
        print('Settings:\n number_of_lines: {}\n batch_size: {}\n weight_loss_transformer: {}\n weight_loss_generator: {}\n n_epochs: {}\n patience: {}'.format(number_of_lines, BATCH_SIZE, weight_loss_transformer, weight_loss_generator, n_epochs, patience))
    elif MMD:
        print('Settings:\n number_of_lines: {}\n batch_size: {}\n weight_loss_transformer: {}\n weight_loss_MMD: {}\n n_epochs: {}\n patience: {}'.format(number_of_lines, BATCH_SIZE, weight_loss_transformer, weight_loss_MMD, n_epochs, patience))
    else:
        print('Settings:\n number_of_lines: {}\n batch_size: {}\n weight_loss_transformer: {}\n n_epochs: {}\n patience: {}'.format(number_of_lines, BATCH_SIZE, weight_loss_transformer, n_epochs, patience))

    # create train and validation set 
    train, val = train_test_split(df, test_size=0.1)
    train.to_csv("train.csv", index=False)
    val.to_csv("val.csv", index=False)

    # associate the text in the 'English' column with the EN_TEXT field, and 'French' with FR_TEXT
    data_fields = [('trg', TGT), ('src', SRC)]
    train,val = data.TabularDataset.splits(path='./', train='train.csv', validation='val.csv', format='csv', fields=data_fields)

    SRC.build_vocab(train.src, val)
    TGT.build_vocab(train.trg, val)
    
print('Vocabulary is built')


Settings:
 number_of_lines: (919200, 4)
 batch_size: 2500
 weight_loss_transformer: 1
 weight_loss_MMD: 10
 n_epochs: 10000
 patience: 100
Vocabulary is built


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ('Available devices {}'.format(torch.cuda.device_count()))

if True:
    pad_idx = TGT.vocab.stoi["<blank>"]
    model = make_model(len(SRC.vocab), len(TGT.vocab), N=6).to(device)
    # model.src_embed.weight = model.src_embed.weight + 0.5
    # model.src_embed.weight = model.src_embed.weight * 0.5
    # with torch.no_grad():
    #     print(model.src_embed[0].lut.weight)
    #     # model.src_embed[0].lut.weight = model.src_embed[0].lut.weight + 0.5
    #     # model.src_embed[0].lut.weight = model.src_embed[0].lut.weight * 0.5
    #     # model.src_embed[0].lut.weight.masked_scatter_(torch.ones_like(model.src_embed[0].lut.weight)>0,model.src_embed[0].lut.weight+0.05)
    #     # model.src_embed[0].lut.weight.masked_scatter_(torch.ones_like(model.src_embed[0].lut.weight)>0,model.src_embed[0].lut.weight*0.05)
    #     # print(model.src_embed[0].lut.weight)
    # with torch.enable_grad():
    #     model.to(device)
    #     print(model.src_embed[0].lut.weight)

    #Save the model just with its initialization 
    best_model_wts = copy.deepcopy(model.state_dict())
    torch.save(best_model_wts, os.path.join(path_to_save_models,string_name + '_epoch0.pth'))

    criterion = LabelSmoothing(size=len(TGT.vocab), padding_idx=pad_idx, smoothing=0.1).to(device)
    # criterion.cuda()
    train_iter = MyIterator(train, batch_size=BATCH_SIZE, device=0,
                            repeat=False, sort_key=lambda x: (len(x.src), len(x.trg)),
                            batch_size_fn=batch_size_fn, train=True)
    valid_iter = MyIterator(val, batch_size=BATCH_SIZE, device=0,
                            repeat=False, sort_key=lambda x: (len(x.src), len(x.trg)),
                            batch_size_fn=batch_size_fn, train=False)
print('Initialized model...')

Available devices 1


NameError: name 'TGT' is not defined

In [14]:
# string_name = 'transformer_GANs_withMMD_1MSent'
model.load_state_dict(torch.load(os.path.join(path_to_save_models,'checkpoint_' + string_name + '.pt')))

<All keys matched successfully>

In [ ]:
training = True
print('Training: {}'.format(training))
if training:
#     del raw_data, df
    torch.cuda.empty_cache()
    train_losses = []
    # to track the validation loss as the model trains
    valid_losses = []
    # to track the average training loss per epoch as the model trains
    avg_train_losses = []
    # to track the average validation loss per epoch as the model trains
    avg_valid_losses = [] 
    # losses_discriminator_train=[]
    losses_transformer_train=[]
    losses_transformer_val=[]
    if EUCLDN:
        losses_generator_train=[]
        losses_generator_val=[]
        # losses_discriminator_val=[]
    elif MMD:
        losses_MMD_train=[]
        losses_MMD_val=[]
    else:
        losses_justTransf_train=[]
        losses_justTransf_val=[]

    # losses_GAN_val=[]
    # losses_GAN_train=[]
    r2_landmarks_val=[]
    # losses_MMD_train=[]
    # losses_MMD_val=[]

    # initialize the early_stopping object
    early_stopping = EarlyStopping(patience=patience, verbose=False)
    # discriminator = DiscriminatorNet()
    # d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.0002)
    # g_optimizer = torch.optim.Adam(model.parameters(), lr=0.0002) #At some point just set g_optimizer = model_opt
    model_opt = NoamOpt(model.src_embed[0].d_model, 1, 2000,
            torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))
    # loss_GAN = torch.nn.MSELoss()

    for epoch in range(1, n_epochs+1):
        print('\nStarting training...')
        model.train()
        # print(model) #The print works, data_gen(v,10,2) works, SimpleLossCompute works
        # print('SRC.vocab.itos.size(0): {}'.format(len(SRC.vocab.itos)))
        # idx_src = np.random.randint(1, len(SRC.vocab.itos), size=(1, 100))
        # idx_tgt = np.random.randint(1, len(TGT.vocab.itos), size=(1, 100))
        # print(SRC.vocab.itos[idx_src])
        # print(sample(SRC.vocab.itos,3))
        # print(len(word_list))
        # print(word_list[0])
        # print(train_iter)
        # for b in train_iter:
        #     print(b)
        #     for j in range(1, batch.trg.size(0)):
        #     sym = TGT.vocab.itos[batch.trg.data[j, 0]]
        #     if sym == "</s>": break
        #     print(sym, end =" ")

        # # Using GANs
        # loss_train, loss_total_train, loss_generator_train, loss_discriminator_train, loss_transformer_train = run_epoch((rebatch(pad_idx, b) for b in train_iter), model, SimpleLossCompute(model.generator, criterion, model.src_embed[0].lut, model.tgt_embed[0].lut, SRC, TGT, discriminator, loss_GAN, english_data_target, model_opt), device=device, SRC=SRC, TGT=TGT, epoch=epoch, n_epochs=n_epochs, logger=logger)
        # print('\nStarting validation... ')
        # model.eval()
        # loss_val,loss_total_val, loss_generator_val, loss_discriminator_val, loss_transformer_val = run_epoch((rebatch(pad_idx, b) for b in valid_iter), model, SimpleLossCompute(model.generator, criterion, model.src_embed[0].lut, model.tgt_embed[0].lut, SRC, TGT, discriminator, loss_GAN, english_data_target, None), device=device, SRC=SRC, TGT=TGT, epoch=epoch, n_epochs=n_epochs, logger=logger)
        # # \Using GANs

        # Using MSE
        # total_loss / total_tokens, loss, total_loss_generator / total_tokens, loss_transformer
        loss_train, loss_total_train, loss_method_train, loss_transformer_train = run_epoch((rebatch(pad_idx, b) for b in train_iter), model, SimpleLossCompute(model.generator, criterion, model.src_embed[0].lut, model.tgt_embed[0].lut, SRC, TGT, mmd, model_opt), device=device, SRC=SRC, TGT=TGT, epoch=epoch, n_epochs=n_epochs)
        print('\nStarting validation... ')
        model.eval()
        loss_val,loss_total_val, loss_method_val, loss_transformer_val = run_epoch((rebatch(pad_idx, b) for b in valid_iter), model, SimpleLossCompute(model.generator, criterion, model.src_embed[0].lut, model.tgt_embed[0].lut, SRC, TGT, mmd, None), device=device, SRC=SRC, TGT=TGT, epoch=epoch, n_epochs=n_epochs)
        
        if EUCLDN:
            loss_generator_train = loss_method_train
            loss_generator_val = loss_method_val
        elif MMD:
            loss_MMD_train = loss_method_train
            loss_MMD_val = loss_method_val
        else:
            loss_justTransf_train = loss_method_train
            loss_justTransf_val = loss_method_val
        # \Using MSE
        # print(loss)

        ### Compute error the landmarks
        src_words_land = word_list.iloc[:,0].tolist()
        tgt_words_land = word_list.iloc[:,1].tolist()
        src_words_land = torch.LongTensor([[SRC.vocab.stoi[w] for w in src_words_land]]) #stoi – A collections.defaultdict instance mapping token strings to numerical identifiers.
        src_words_land = Variable(src_words_land)
        tgt_words_land = torch.LongTensor([[TGT.vocab.stoi[w] for w in tgt_words_land]]) #stoi – A collections.defaultdict instance mapping token strings to numerical identifiers.
        tgt_words_land = Variable(tgt_words_land)
        embed_src = model.src_embed[0].lut(src_words_land.to(device)).squeeze()
        embed_tgt = model.tgt_embed[0].lut(tgt_words_land.to(device)).squeeze()

        if epoch % 20 ==0:
            plot_embeddings(path_save,string_name, epoch)


        mean_r2=0
        # print("embed_src.size(): {}".format(embed_src.size()))
        for j in range(1,embed_src.size(0)):
            # mean_r2 += r2_score(embed_src.cpu().detach().numpy(), embed_tgt.cpu().detach().numpy())
            mean_r2 += (np.corrcoef(embed_src[j,:].cpu().detach().numpy(), embed_tgt[j,:].cpu().detach().numpy(), rowvar=False)[0,1])**2
        r2_landmarks = mean_r2/embed_src.size(0)

        r2_landmarks_val.append(r2_landmarks)
        train_losses.append(loss_train)
        valid_losses.append(loss_val)
        losses_transformer_train.append(loss_transformer_train)
        losses_transformer_val.append(loss_transformer_val)
        if EUCLDN:
            losses_generator_train.append(loss_generator_train)
            # losses_discriminator_train.append(loss_discriminator_train)
            # losses_discriminator_val.append(loss_discriminator_val)
            losses_generator_val.append(loss_generator_val)
        elif MMD:
            losses_MMD_train.append(loss_MMD_train)
            losses_MMD_val.append(loss_MMD_val)
        # losses_GAN_train.append(loss_GAN_train)
        # losses_GAN_val.append(loss_GAN_val)
        else:
            losses_justTransf_train.append(loss_justTransf_train)
            losses_justTransf_val.append(loss_justTransf_val)


        train_loss = np.average(train_losses) #/ (len(dataloaders['train'].dataset))
        valid_loss = np.average(valid_losses) #/ (len(dataloaders['val'].dataset))
        avg_train_losses.append(train_loss)
        avg_valid_losses.append(valid_loss)

        epoch_len = len(str(n_epochs))

        print_msg = (f'[{epoch:>{epoch_len}}/{n_epochs:>{epoch_len}}] ' +
                 f'train_loss: {loss_train:.5f} ' +
                 f'valid_loss: {loss_val:.5f} ' +
                 f'landmarks_r2: {r2_landmarks:.5f}')
        print(print_msg)

         # Log batch error

        # print('validation_loss: {}\n model: {}\n structure_net: {}'.format(valid_loss,model,structure_net))
        # string_name = 'transformer_v6'
        early_stopping(loss_val, model, string_name, path_to_save_models)
#         early_stopping(loss_val, model, string_name)
        # clear lists to track next epoch
        # train_losses = []
        # valid_losses = []

        # print(model.src_embed[0].lut.weight)
        if EUCLDN:
            fig, ax = plt.subplots(figsize=(15, 16), dpi= 80, facecolor='w', edgecolor='k')
            ax.plot(range(1,epoch+1), train_losses, '-o')
            ax.plot(range(1,epoch+1), valid_losses, '-o')
            ax.set(xlabel='epoch', ylabel='Loss', title='Total losses (generator+transformer)')
            ax.legend(['train_losses','valid_losses'])
            plt.savefig(os.path.join(path_to_save_plots,string_name+'_losses_total_'+ string_name + '.png'))
        elif MMD:
            fig, ax = plt.subplots(figsize=(15, 16), dpi= 80, facecolor='w', edgecolor='k')
            ax.plot(range(1,epoch+1), train_losses, '-o')
            ax.plot(range(1,epoch+1), valid_losses, '-o')
            ax.set(xlabel='epoch', ylabel='Loss', title='Total losses (MMD+transformer)')
            ax.legend(['train_losses','valid_losses'])
            plt.savefig(os.path.join(path_to_save_plots,string_name + '_losses_total_'+ string_name + '.png'))
        else:
            fig, ax = plt.subplots(figsize=(15, 16), dpi= 80, facecolor='w', edgecolor='k')
            ax.plot(range(1,epoch+1), train_losses, '-o')
            ax.plot(range(1,epoch+1), valid_losses, '-o')
            ax.set(xlabel='epoch', ylabel='Loss', title='Total losses (transformer)')
            ax.legend(['train_losses','valid_losses'])
            plt.savefig(os.path.join(path_to_save_plots,string_name + '_losses_total_'+ string_name + '.png'))

        fig, ax = plt.subplots(figsize=(15, 16), dpi= 80, facecolor='w', edgecolor='k')
        ax.plot(range(1,epoch+1), losses_transformer_train, '-o')
        ax.plot(range(1,epoch+1), losses_transformer_val, '-o')
        ax.set(xlabel='epoch', ylabel='Loss', title='Transformer loss')
        ax.legend(['losses_transformer_train','losses_transformer_val'])
        plt.savefig(os.path.join(path_to_save_plots,string_name + '_losses_transformer_'+ string_name + '.png'))

        # fig, ax = plt.subplots(figsize=(15, 16), dpi= 80, facecolor='w', edgecolor='k')
        # ax.plot(range(1,epoch+1), losses_GAN_train, '-o')
        # ax.plot(range(1,epoch+1), losses_GAN_val, '-o')
        # ax.set(xlabel='epoch', ylabel='Loss', title='GAN loss')
        # ax.legend(['losses_GAN_train','losses_GAN_val'])
        # plt.savefig('losses_GAN.png')

        # fig, ax = plt.subplots(figsize=(15, 16), dpi= 80, facecolor='w', edgecolor='k')
        # ax.plot(range(1,epoch+1), losses_discriminator_train, '-o')
        # ax.plot(range(1,epoch+1), losses_discriminator_val, '-o')
        # ax.set(xlabel='epoch', ylabel='Loss', title='Discriminator loss')
        # ax.legend(['losses_discriminator_train', 'losses_discriminator_val'])
        # plt.savefig('losses_discriminator.png')
        if EUCLDN:
            fig, ax = plt.subplots(figsize=(15, 16), dpi= 80, facecolor='w', edgecolor='k')
            ax.plot(range(1,epoch+1), losses_generator_train, '-o')
            ax.plot(range(1,epoch+1), losses_generator_val, '-o')
            ax.set(xlabel='epoch', ylabel='Loss', title='Generator loss')
            ax.legend(['losses_generator_train', 'losses_generator_val'])
            plt.savefig('losses_generator_'+ string_name + '.png')
        elif MMD:
            fig, ax = plt.subplots(figsize=(15, 16), dpi= 80, facecolor='w', edgecolor='k')
            ax.plot(range(1,epoch+1), losses_MMD_train, '-o')
            ax.plot(range(1,epoch+1), losses_MMD_val, '-o')
            ax.set(xlabel='epoch', ylabel='Loss', title='MMD loss')
            ax.legend(['losses_MMD_train','losses_MMD_val'])
            plt.savefig('losses_MMD_'+ string_name + '.png')
        else:
            fig, ax = plt.subplots(figsize=(15, 16), dpi= 80, facecolor='w', edgecolor='k')
            ax.plot(range(1,epoch+1), losses_justTransf_train, '-o')
            ax.plot(range(1,epoch+1), losses_justTransf_val, '-o')
            ax.set(xlabel='epoch', ylabel='Loss', title='Just Transformer')
            ax.legend(['losses_transf_train','losses_transf_val'])
            plt.savefig(os.path.join(path_to_save_plots,string_name + '_losses_transf_'+ string_name + '.png'))

        fig, ax = plt.subplots(figsize=(15, 16), dpi= 80, facecolor='w', edgecolor='k')
        ax.plot(range(1,epoch+1), r2_landmarks_val, '-o')
        ax.set(xlabel='epoch', ylabel='R^2', title='Landmarks R_2')
        ax.legend(['r2_landmarks_val'])
        plt.savefig(os.path.join(path_to_save_plots,string_name +'_r2_landmarks_'+ string_name + '.png'))

        plt.close('all')

        if early_stopping.early_stop:
            print("Early stopping")
            break

        if epoch == 1:
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(best_model_wts, os.path.join(path_to_save_models,string_name + '_epoch1.pth'))
        if epoch == n_epochs:
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(best_model_wts, os.path.join(path_to_save_models,string_name + '_lastEpoch.pth'))

    #save the model
    model.load_state_dict(torch.load(os.path.join(path_to_save_models,'checkpoint_' + string_name + '.pt')))
    best_model_wts = copy.deepcopy(model.state_dict())
    torch.save(best_model_wts, os.path.join(path_to_save_models,'best_model_' + string_name + '_transformers.pth'))
else:
    # model = torch.load("iwslt.pt") #For some reason it is not working very well
#     model.load_state_dict(torch.load('best_model_transformers_v4.pth'))
    model.load_state_dict(torch.load(os.path.join(path_to_save_models,'checkpoint_' + string_name + '.pt')))


Training: True

Starting training...


/opt/conda/conda-bld/pytorch_1587428398394/work/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


Epoch Step: 0 Loss: 3.9463 (loss_MMD: 1.2712, loss_trans: 2.6751) Tokens per Sec: 781.6782
Epoch Step: 100 Loss: 4.6588 (loss_MMD: 2.1698, loss_trans: 2.4891) Tokens per Sec: 2735.7631
Epoch Step: 200 Loss: 3.6180 (loss_MMD: 0.8894, loss_trans: 2.7286) Tokens per Sec: 2731.8367
Epoch Step: 300 Loss: 5.5556 (loss_MMD: 3.1372, loss_trans: 2.4183) Tokens per Sec: 3107.3412
Epoch Step: 400 Loss: 5.8826 (loss_MMD: 2.9409, loss_trans: 2.9417) Tokens per Sec: 3023.1330
Epoch Step: 500 Loss: 4.4675 (loss_MMD: 1.6219, loss_trans: 2.8456) Tokens per Sec: 3073.9396
Epoch Step: 600 Loss: 4.6854 (loss_MMD: 1.4618, loss_trans: 3.2236) Tokens per Sec: 3163.5795
Epoch Step: 700 Loss: 4.9561 (loss_MMD: 1.7663, loss_trans: 3.1898) Tokens per Sec: 3048.8676
Epoch Step: 800 Loss: 4.7286 (loss_MMD: 1.8183, loss_trans: 2.9102) Tokens per Sec: 2973.4853
Epoch Step: 900 Loss: 5.0742 (loss_MMD: 2.4711, loss_trans: 2.6031) Tokens per Sec: 3094.8463
Epoch Step: 1000 Loss: 9.0393 (loss_MMD: 6.5512, loss_trans: 2.

In [ ]:
show_validationSet = True
if show_validationSet:
    for i, batch in enumerate(valid_iter):
        print('Value of i: {}'.format(i))
        src = batch.src.transpose(0, 1)[:1]
        # src = batch.src.transpose(0, 1)[2]
        # print('src: {}'.format(src))
        # src = src.cuda()
        src_mask = (src != SRC.vocab.stoi["<blank>"]).unsqueeze(-2)
        # src_mask = src_mask.cuda()
        # print('src_mask: {}; src: {}'.format(src_mask.is_cuda, src.is_cuda))
        out = greedy_decode(model.to(device), src.to(device), src_mask.to(device), max_len=60, start_symbol=TGT.vocab.stoi["<s>"])
        print("Translation:", end="\t")
        for j in range(1, out.size(1)):
            sym = TGT.vocab.itos[out[0, j]]
            if sym == "</s>": break
            print(sym, end =" ")
        print()
        print("Target:", end="\t")
        for j in range(1, batch.trg.size(0)):
            sym = TGT.vocab.itos[batch.trg.data[j, 0]]
            if sym == "</s>": break
            print(sym, end =" ")
        print()
        if i==10:
            break

In [26]:
######## Real world example 2 ####### not working
# model, SRC, TGT = torch.load("en-de-model.pt") #not working... seems to be associated to a bug in pytorch: http://forum.opennmt.net/t/issue-loading-pretrained-models-for-machine-translation/2982
show_mySentences = True
if show_mySentences:
    print("\nTranslating my samples:")
    model.eval()
    sent = "Je suis si heureux de faire partie de ce groupe et vous allez tous me manque.".split()
    src = torch.LongTensor([[SRC.vocab.stoi[w] for w in sent]]) #stoi – A collections.defaultdict instance mapping token strings to numerical identifiers.
    src = Variable(src)
    src_mask = (src != SRC.vocab.stoi["<blank>"]).unsqueeze(-2)
    out = greedy_decode(model.to(device), src.to(device), src_mask.to(device), max_len=60, start_symbol=TGT.vocab.stoi["<s>"])

    print("Translation:", end="\t")
    trans = "<s> "
    for i in range(1, out.size(1)):
        sym = TGT.vocab.itos[out[0, i]] #itos – A list of token strings indexed by their numerical identifiers.
        if sym == "</s>": break
        trans += sym + " "
    print(trans)

    sent = "Les enfants courent dehors pendant que le chien aboie.".split()
    src = torch.LongTensor([[SRC.vocab.stoi[w] for w in sent]])
    src = Variable(src)
    src_mask = (src != SRC.vocab.stoi["<blank>"]).unsqueeze(-2)
    out = greedy_decode(model.to(device), src.to(device), src_mask.to(device), max_len=60, start_symbol=TGT.vocab.stoi["<s>"])

    print("Translation:", end="\t")
    trans = "<s> "
    for i in range(1, out.size(1)):
        sym = TGT.vocab.itos[out[0, i]]
        if sym == "</s>": break
        trans += sym + " "
    print(trans)

    sent = "Il fait trop froid pour faire du vélo.".split()
    src = torch.LongTensor([[SRC.vocab.stoi[w] for w in sent]])
    src = Variable(src)
    src_mask = (src != SRC.vocab.stoi["<blank>"]).unsqueeze(-2)
    out = greedy_decode(model.to(device), src.to(device), src_mask.to(device), max_len=60, start_symbol=TGT.vocab.stoi["<s>"])

    print("Translation:", end="\t")
    trans = "<s> "
    for i in range(1, out.size(1)):
        sym = TGT.vocab.itos[out[0, i]]
        if sym == "</s>": break
        trans += sym + " "
    print(trans)

    #Print attention
    tgt_sent = trans.split()
    def draw(data, x, y, ax):
        seaborn.heatmap(data, 
                        xticklabels=x, square=True, yticklabels=y, vmin=0.0, vmax=1.0, 
                        cbar=False, ax=ax)
        
#     for layer in range(1, 6, 2):
#         fig, axs = plt.subplots(1,4, figsize=(20, 10))
#         print("Encoder Layer", layer+1)
#         for h in range(4):
#             draw(model.encoder.layers[layer].self_attn.attn[0, h].data.cpu(), sent, sent if h ==0 else [], ax=axs[h])
#         plt.show()
#         fig.suptitle('enc_layer_' + str(layer+1) , fontsize=16)
#         plt.savefig('enc_layer_' + str(layer+1) + '.png')
        
#     for layer in range(1, 6, 2):
#         fig, axs = plt.subplots(1,4, figsize=(20, 10))
#         print("Decoder Self Layer", layer+1)
#         for h in range(4):
#             draw(model.decoder.layers[layer].self_attn.attn[0, h].data[:len(tgt_sent), :len(tgt_sent)].cpu(), 
#                 tgt_sent, tgt_sent if h ==0 else [], ax=axs[h])
#         plt.show()
#         fig.suptitle('dec_self_layer_' + str(layer+1) , fontsize=16)
#         plt.savefig('dec_self_layer_' + str(layer+1) + '.png')
#         print("Decoder Src Layer", layer+1)
#         fig, axs = plt.subplots(1,4, figsize=(20, 10))
#         for h in range(4):
#             draw(model.decoder.layers[layer].self_attn.attn[0, h].data[:len(tgt_sent), :len(sent)].cpu(), 
#                 sent, tgt_sent if h ==0 else [], ax=axs[h])
#         plt.show()
#         fig.suptitle('dec_src_layer_' + str(layer+1) , fontsize=16)
#         plt.savefig('dec_src_layer_' + str(layer+1) + '.png')

######## \Real world example 2 #######


Translating my samples:
Translation:	<s> I am pleased to see that the group of the Party of European Socialists is not in favour of this . 
Translation:	<s> The children are not , however , a word of communication . 
Translation:	<s> It is not enough to be able to do so . 


In [27]:
##### Plotting the word embedding #####
if False: 
    for i, batch in enumerate(valid_iter):
        print('Value of i: {}'.format(i))
        sent = "Je suis si heureux de faire partie de ce groupe et vous allez tous me manque.".split()
        src = torch.LongTensor([[SRC.vocab.stoi[w] for w in sent]]) #stoi – A collections.defaultdict instance mapping token strings to numerical identifiers.
        src = Variable(src)

        print('src.size(): {}'.format(src.size()))
        # print('src_embed.size(): {}'.format(src_embed.size()))
        # out_emb = model.src_embed[0]
        # print(model)
        # print(out_emb)
        # out_pos = model.src_embed[1]
        # print(out_pos)
        out_emb1 = model.src_embed[0].lut
        print(out_emb1)

        test = out_emb1(src.to(device))
        print('test: {}'.format(test))
        break

In [28]:
if True: 
    print('\nShowing word embedding (whole vocab)...')
    string_name = 'transformer_v6'
    for i, batch in enumerate(valid_iter):
        print('Value of i: {}'.format(i))
        # sent = "A veteran is a person who has had long service or experience in a particular occupation or field in the war.".split()
        # print('TGT.vocab: {}'.format(TGT.vocab))
        sent = TGT.vocab.itos
        src = torch.LongTensor([[TGT.vocab.stoi[w] for w in sent]]) #stoi – A collections.defaultdict instance mapping token strings to numerical identifiers.
        src = Variable(src)
        print('src: {}'.format(src))
        print('src.size(): {}'.format(src.size()))
        # print('src_embed.size(): {}'.format(src_embed.size()))
        # out_emb = model.src_embed[0]
        # print(model)
        # print(out_emb)
        # out_pos = model.src_embed[1]
        # print(out_pos)
        out_emb1 = model.tgt_embed[0].lut
        print(out_emb1)

        test = out_emb1(src.to(device))
        print('test.size(): {}'.format(test.size()))
        print('test: {}'.format(test))
        torch.save(test, 'tgt_embedded.pt')
        break


Showing word embedding (whole vocab)...
Value of i: 0
src: tensor([[    0,     1,     2,  ..., 30334, 30335, 30336]])
src.size(): torch.Size([1, 30337])
Embedding(30337, 512)
test.size(): torch.Size([1, 30337, 512])
test: tensor([[[-0.0065,  0.0025, -0.0068,  ..., -0.0033, -0.0004, -0.0022],
         [ 0.0116, -0.0012,  0.0097,  ..., -0.0056,  0.0099, -0.0108],
         [-0.0260, -0.0809, -0.1730,  ...,  0.0735,  0.0066,  0.0648],
         ...,
         [-0.0078, -0.0260,  0.0096,  ..., -0.0114,  0.0224, -0.0055],
         [ 0.0219, -0.0044,  0.0106,  ..., -0.0072,  0.0422, -0.0093],
         [ 0.0206, -0.0085,  0.0053,  ...,  0.0057, -0.0082, -0.0219]]],
       device='cuda:0', grad_fn=<EmbeddingBackward>)


In [29]:
# #### PHATE pot for the TGT (english) embedding
# print('\nEmbedding (from 512 to 2 or 3) with PHATE...')
# gamma=0
# phate_operator = phate.PHATE(n_components=2, k=5, a=20, t=10, gamma=gamma, n_jobs=-1)
# test = test.squeeze()
# n_samples = 2000
# data_phate = phate_operator.fit_transform(test[:n_samples].cpu().detach().numpy())
# torch.save(data_phate, 'data_phate_2D.pt')

# ### Plot the 2D embedding (just a couple of samples)###
# #%matplotlib notebook
# fig, ax = plt.subplots(figsize=(15, 16), dpi= 80, facecolor='w', edgecolor='k')
# ax.scatter(data_phate[:n_samples,0], data_phate[:n_samples,1], s=20)
# for i in range(0,n_samples-1):
#     ax.annotate(sent[i], (data_phate[i,0], data_phate[i,1]), fontsize=10)

# # %%
# if True: 
#     string_name = 'transformer_v6'
#     for i, batch in enumerate(valid_iter):
#         print('Value of i: {}'.format(i))
#         # sent = "A veteran is a person who has had long service or experience in a particular occupation or field in the war.".split()
#         # print('TGT.vocab: {}'.format(TGT.vocab))
#         sent = SRC.vocab.itos
#         src = torch.LongTensor([[SRC.vocab.stoi[w] for w in sent]]) #stoi – A collections.defaultdict instance mapping token strings to numerical identifiers.
#         src = Variable(src)
#         print('src: {}'.format(src))
#         print('src.size(): {}'.format(src.size()))
#         # print('src_embed.size(): {}'.format(src_embed.size()))
#         # out_emb = model.src_embed[0]
#         # print(model)
#         # print(out_emb)
#         # out_pos = model.src_embed[1]
#         # print(out_pos)
#         out_emb1 = model.src_embed[0].lut
#         print(out_emb1)

#         test = out_emb1(src.to(device))
#         print('test.size(): {}'.format(test.size()))
#         print('test: {}'.format(test))
#         torch.save(test, 'src_embedded.pt')
#         break

# # %%
# #### PHATE pot for the TGT (english) embedding
# print('\nEmbedding (from 512 to 2 or 3) with PHATE...')
# gamma=0
# phate_operator = phate.PHATE(n_components=2, k=5, a=20, t=10, gamma=gamma, n_jobs=-1)
# test = test.squeeze()
# n_samples = 2000
# data_phate = phate_operator.fit_transform(test[:n_samples].cpu().detach().numpy())
# torch.save(data_phate, 'data_phate_src_2D.pt')

# ### Plot the 2D embedding (just a couple of samples)###
# #%matplotlib notebook
# fig, ax = plt.subplots(figsize=(15, 16), dpi= 80, facecolor='w', edgecolor='k')
# ax.scatter(data_phate[:n_samples,0], data_phate[:n_samples,1], s=20)
# for i in range(0,n_samples-1):
#     ax.annotate(sent[i], (data_phate[i,0], data_phate[i,1]), fontsize=10)

# # %%


# # # # print(labels_all)
# # fig1 = scprep.plot.scatter2d(data_phate, filename= string_name + ".png",title=string_name, ticks=False, label_prefix="PHATE")
# # # fig2 = scprep.plot.rotate_scatter3d(data_phate, filename=string_name + ".gif", title=string_name)


# # phate_operator = phate.PHATE(n_components=3, k=5, a=20, t=150, gamma=gamma, n_jobs=-1)
# # # test = test.squeeze()
# # data_phate = phate_operator.fit_transform(test.cpu().detach().numpy())
# # # print('data_phate shape: {}\n'.format(data_phate.shape))
# # torch.save(data_phate, 'data_phate_3D.pt')

# # # # print(labels_all)
# # # fig1 = scprep.plot.scatter2d(data_phate, filename= string_name + ".png", title=string_name, ticks=False, label_prefix="PHATE")
# # fig2 = scprep.plot.rotate_scatter3d(data_phate, filename=string_name + ".gif", title=string_name)

# # plt.clf()
# # plt.close()
# # # gc.collect()

# # del fig1, fig2

# # del data_phate, phate_operator
# # torch.cuda.empty_cache()

# # # summary(model, (src, src_mask))

# # %%
# if True: 
#     string_name = 'transformer_v6'
#     for i, batch in enumerate(valid_iter):
#         print('Value of i: {}'.format(i))
#         # sent = "A veteran is a person who has had long service or experience in a particular occupation or field in the war.".split()
#         # print('TGT.vocab: {}'.format(TGT.vocab))
#         sent = SRC.vocab.itos[:2000]
#         src = torch.LongTensor([[SRC.vocab.stoi[w] for w in sent]]) #stoi – A collections.defaultdict instance mapping token strings to numerical identifiers.
#         src = Variable(src)
#         print('src: {}'.format(src))
#         print('src.size(): {}'.format(src.size()))
#         # print('src_embed.size(): {}'.format(src_embed.size()))
#         # out_emb = model.src_embed[0]
#         # print(model)
#         # print(out_emb)
#         # out_pos = model.src_embed[1]
#         # print(out_pos)
#         out_emb1 = model.src_embed[0].lut
#         print(out_emb1)

#         test = out_emb1(src.to(device))
#         print('test.size(): {}'.format(test.size()))
#         print('test: {}'.format(test))
#         torch.save(test, 'src_embedded.pt')
        
#         print('Value of i: {}'.format(i))
#         # sent = "A veteran is a person who has had long service or experience in a particular occupation or field in the war.".split()
#         # print('TGT.vocab: {}'.format(TGT.vocab))
#         sent = SRC.vocab.itos[:2000]
#         src = torch.LongTensor([[SRC.vocab.stoi[w] for w in sent]]) #stoi – A collections.defaultdict instance mapping token strings to numerical identifiers.
#         src = Variable(src)
#         print('src: {}'.format(src))
#         print('src.size(): {}'.format(src.size()))
#         # print('src_embed.size(): {}'.format(src_embed.size()))
#         # out_emb = model.src_embed[0]
#         # print(model)
#         # print(out_emb)
#         # out_pos = model.src_embed[1]
#         # print(out_pos)
#         out_emb1 = model.src_embed[0].lut
#         print(out_emb1)

#         test = out_emb1(src.to(device))
#         print('test.size(): {}'.format(test.size()))
#         print('test: {}'.format(test))
#         torch.save(test, 'src_embedded.pt')
#         break
